In [1]:
# Import packages
import requests
import json
import pandas as pd





In [2]:
#make a request of the UN SDG API to get series with data availability over 5 years
# https://unstats.un.org/SDGAPI/swagger/#!/DataAvailability/V1SdgDataAvailabilityGetSeriesAggregationsForMapsPost
url =  "https://unstats.un.org/SDGAPI/v1/sdg/DataAvailability/GetSeriesAggregationsForMaps"

r=requests.post(
  url=url,
  data={
    'dataPointType':'1',
    'natureOfData':'All',
    'goalId':'1'
  }
)


In [15]:
# turn into pandas dataframe
urlData = r.json()
#df=pd.DataFrame.from_dict(urlData)

df = pd.json_normalize(urlData, record_path='indicators',
                                        meta=['index','name','percentage', 'tier'],
                                        errors='ignore',
                                        record_prefix="indicator_")

sdg_df.rename(columns={'name':'iso3c', 'percentage':'goal_percentage'}, inplace=True)                                        

In [24]:
# get ISO3c codes
iso3c = pd.read_csv('https://raw.githubusercontent.com/worldbank/SPI/master/01_raw_data/metadata/iso_codes.csv')

span = range(2005,2021)

print(span)

iso_empty=iso3c.copy()
iso_empty['date']=2004

for i in span:
    print(i)
    temp=iso3c.copy()
    temp['date']=i
    iso_empty=iso_empty.append(temp)

count=iso_empty.groupby('date').count()

range(2005, 2021)
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [2]:
# Construct the URL for the API call
URL     = "https://unstats.un.org/SDGAPI/v1/sdg/Indicator/List"

# Submit the API data request
data = requests.get(URL).json()

# Convert the JSON data to a pandas data frame
indicator_metadata  = pd.read_json(json.dumps(data))
indicator_metadata = pd.json_normalize(data, record_path='series',
                                        meta=['goal','target','code','description', 'tier'],
                                        errors='ignore',
                                        record_prefix="m_")


indicator_metadata_t1 = indicator_metadata[indicator_metadata.tier=='1']

series_codes=indicator_metadata['m_code'].tolist()

In [39]:
#fetch data from API
URL = 'https://unstats.un.org/SDGAPI/v1/sdg/Series/Data?seriesCode='+series_codes[0]+'&pageSize=50000'
sdg_json = requests.get(URL).json()

#clean up json
sdg_df_raw=pd.json_normalize(sdg_json, record_path='data')
#rename columns to match
sdg_df=sdg_df_raw.copy()
sdg_df.rename(columns={'geoAreaCode':'iso3c', 'timePeriodStart':'date'}, inplace=True)
sdg_df=sdg_df[['iso3c','date','goal', 'target', 'indicator', 'series', 'seriesDescription','value',  
            'source', 'geoInfoUrl', 'footnotes', 'attributes.Nature','attributes.Units', 'dimensions.Reporting Type']]

pd.merge(sdg_df,iso_empty,how='right',
               on=['iso3c','date'])


ChunkedEncodingError: ("Connection broken: ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)", ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))